Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
batch_size = 128


print('train_dataset' + str(train_dataset.shape))
print('train_labels' + str(train_labels.shape))
print('valid_dataset' + str(valid_dataset.shape))
print('test_dataset' + str(test_dataset.shape))
# print('weights' + str(weights.shape))
# print('biases' + str(biases.shape))
# print('logits' + str(logits.shape))

num_nodes_hidden_layer = 1024


reluGraph = tf.Graph()
with reluGraph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # variables for both steps
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes_hidden_layer]))
    biases1 = tf.Variable(tf.zeros([num_nodes_hidden_layer]))

    weights2 = tf.Variable(tf.truncated_normal([num_nodes_hidden_layer, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # first step + relu
    inter_result = tf.matmul(tf_train_dataset, weights1) + biases1
    post_relu = tf.nn.relu(features=inter_result)
    
    # second step
    logits = tf.matmul(post_relu, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
                          + 0.01*tf.nn.l2_loss(weights1) + 0.01*tf.nn.l2_loss(weights2))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_inter_step = tf.nn.relu(features=(tf.matmul(tf_valid_dataset, weights1) + biases1))
    valid_prediction = tf.nn.softmax(tf.matmul(valid_inter_step, weights2) + biases2)
    
    test_inter_step = tf.nn.relu(features=(tf.matmul(tf_test_dataset, weights1) + biases1))
    test_prediction = tf.nn.softmax(tf.matmul(test_inter_step, weights2) + biases2)
    
    
num_steps = 3001

with tf.Session(graph=reluGraph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

train_dataset(200000, 784)
train_labels(200000, 10)
valid_dataset(10000, 784)
test_dataset(10000, 784)
Initialized
Minibatch loss at step 0: 3530.876953
Minibatch accuracy: 7.0%
Validation accuracy: 16.4%
Minibatch loss at step 500: 21.447777
Minibatch accuracy: 81.2%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 1.105318
Minibatch accuracy: 77.3%
Validation accuracy: 83.8%
Minibatch loss at step 1500: 0.741932
Minibatch accuracy: 82.8%
Validation accuracy: 84.2%
Minibatch loss at step 2000: 0.841146
Minibatch accuracy: 76.6%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 0.813793
Minibatch accuracy: 80.5%
Validation accuracy: 83.6%
Minibatch loss at step 3000: 0.637648
Minibatch accuracy: 85.9%
Validation accuracy: 84.2%
Test accuracy: 90.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [20]:
batch_size = 128


print('train_dataset' + str(train_dataset.shape))
print('train_labels' + str(train_labels.shape))
print('valid_dataset' + str(valid_dataset.shape))
print('test_dataset' + str(test_dataset.shape))
# print('weights' + str(weights.shape))
# print('biases' + str(biases.shape))
# print('logits' + str(logits.shape))

num_nodes_hidden_layer = 1024


reluGraph = tf.Graph()
with reluGraph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # variables for both steps
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes_hidden_layer]))
    biases1 = tf.Variable(tf.zeros([num_nodes_hidden_layer]))

    weights2 = tf.Variable(tf.truncated_normal([num_nodes_hidden_layer, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # first step + relu
    inter_result = tf.matmul(tf_train_dataset, weights1) + biases1
    post_relu = tf.nn.relu(features=inter_result)
    
    # second step
    logits = tf.matmul(post_relu, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
                          + 0.01*tf.nn.l2_loss(weights1) + 0.01*tf.nn.l2_loss(weights2))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_inter_step = tf.nn.relu(features=(tf.matmul(tf_valid_dataset, weights1) + biases1))
    valid_prediction = tf.nn.softmax(tf.matmul(valid_inter_step, weights2) + biases2)
    
    test_inter_step = tf.nn.relu(features=(tf.matmul(tf_test_dataset, weights1) + biases1))
    test_prediction = tf.nn.softmax(tf.matmul(test_inter_step, weights2) + biases2)
    
    
num_steps = 6

with tf.Session(graph=reluGraph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

train_dataset(200000, 784)
train_labels(200000, 10)
valid_dataset(10000, 784)
test_dataset(10000, 784)
Initialized
Minibatch loss at step 0: 3473.801758
Minibatch accuracy: 7.0%
Validation accuracy: 29.8%
Test accuracy: 68.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [17]:
batch_size = 128


print('train_dataset' + str(train_dataset.shape))
print('train_labels' + str(train_labels.shape))
print('valid_dataset' + str(valid_dataset.shape))
print('test_dataset' + str(test_dataset.shape))
# print('weights' + str(weights.shape))
# print('biases' + str(biases.shape))
# print('logits' + str(logits.shape))

num_nodes_hidden_layer = 1024
keep_prob = tf.placeholder(tf.float32)
beta = 0.001

reluGraph = tf.Graph()
with reluGraph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    keep_prob = tf.placeholder(tf.float32)
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # variables for both steps
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes_hidden_layer]))
    hidden_biases = tf.Variable(tf.zeros([num_nodes_hidden_layer]))

    out_weights = tf.Variable(tf.truncated_normal([num_nodes_hidden_layer, num_labels]))
    out_biases = tf.Variable(tf.zeros([num_labels]))
    
    
    # first step + relu
    hidden_layer = tf.nn.relu(features=(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases))
    
    hidden_layer_dropout = tf.nn.dropout(hidden_layer, keep_prob)
    
    # second step
    logits = tf.matmul(hidden_layer_dropout, out_weights) + out_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
                          + beta*tf.nn.l2_loss(hidden_weights) 
                          + beta*tf.nn.l2_loss(out_weights))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_inter_step = tf.nn.relu(features=(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases))
    valid_prediction = tf.nn.softmax(tf.matmul(valid_inter_step, out_weights) + out_biases)
    
    test_inter_step = tf.nn.relu(features=(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases))
    test_prediction = tf.nn.softmax(tf.matmul(test_inter_step, out_weights) + out_biases)
    
    
num_steps = 3001
print('num steps: ' + str(num_steps))

with tf.Session(graph=reluGraph) as session:
  tf.global_variables_initializer().run()
  print("Initialized - " + str(num_steps))
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

train_dataset(200000, 784)
train_labels(200000, 10)
valid_dataset(10000, 784)
test_dataset(10000, 784)
num steps: 3001
Initialized - 3001
Minibatch loss at step 0: 773.392395
Minibatch accuracy: 11.7%
Validation accuracy: 15.8%
Minibatch loss at step 500: 206.499298
Minibatch accuracy: 73.4%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 123.855103
Minibatch accuracy: 72.7%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 71.029373
Minibatch accuracy: 74.2%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 42.253700
Minibatch accuracy: 73.4%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 25.603386
Minibatch accuracy: 76.6%
Validation accuracy: 85.1%
Minibatch loss at step 3000: 15.400259
Minibatch accuracy: 88.3%
Validation accuracy: 85.4%
Test accuracy: 91.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [14]:
batch_size = 128
beta = 0.001

print('train_dataset' + str(train_dataset.shape))
print('train_labels' + str(train_labels.shape))
print('valid_dataset' + str(valid_dataset.shape))
print('test_dataset' + str(test_dataset.shape))
# print('weights' + str(weights.shape))
# print('biases' + str(biases.shape))
# print('logits' + str(logits.shape))

num_nodes_hidden_layer_1 = 1024
num_nodes_hidden_layer_2 = 512


reluGraph = tf.Graph()
with reluGraph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size), name="train_dataset")
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name="train_labels")
    tf_valid_dataset = tf.constant(valid_dataset, name="valid_dataset")
    tf_test_dataset = tf.constant(test_dataset, name="test_dataset")
  
    # first layer
    hidden_weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes_hidden_layer_1]), name="hidden_weights_1")
    hidden_biases_1 = tf.Variable(tf.zeros([num_nodes_hidden_layer_1]), name="hidden_biases_1")
    hidden_layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights_1) + hidden_biases_1, name="hidden_layer_1")
    
    keep_prob = tf.placeholder(tf.float32)
    hidden_layer_1_dropout = tf.nn.dropout(hidden_layer_1, keep_prob)
    
    # second layer
    hidden_weights_2 = tf.Variable(tf.truncated_normal([num_nodes_hidden_layer_1, num_nodes_hidden_layer_2]), name="hidden_weights_2")
    hidden_biases_2 = tf.Variable(tf.zeros([num_nodes_hidden_layer_2]), name="hidden_biases_2")
    
    hidden_layer_2 = tf.nn.relu(tf.matmul(hidden_layer_1_dropout, hidden_weights_2) + hidden_biases_2, name="hidden_layer_2")
    hidden_layer_2_dropout = tf.nn.dropout(hidden_layer_2, keep_prob)
    
    # variables
    out_weights = tf.Variable(tf.truncated_normal([num_nodes_hidden_layer_2, num_labels]), name="out_weights")
    out_biases = tf.Variable(tf.zeros([num_labels]), name="out_biases")
    
    # third step
    logits = tf.matmul(hidden_layer_2_dropout, out_weights) + out_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    loss = tf.reduce_mean( loss + 
                          beta * (tf.nn.l2_loss(out_weights) +
                                  tf.nn.l2_loss(hidden_weights_2) +
                                  tf.nn.l2_loss(hidden_weights_1)))
    
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learnr = tf.placeholder("float")
    learning_rate = tf.train.exponential_decay(learnr, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_inter_step_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights_1) + hidden_biases_1)
    valid_inter_step_2 = tf.nn.relu(tf.matmul(valid_inter_step_1, hidden_weights_2) + hidden_biases_2)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_inter_step_2, out_weights) + out_biases)
    
    test_inter_step_1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights_1) + hidden_biases_1)
    test_inter_step_2 = tf.nn.relu(tf.matmul(test_inter_step_1, hidden_weights_2) + hidden_biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(test_inter_step_2, out_weights) + out_biases)


train_dataset(200000, 784)
train_labels(200000, 10)
valid_dataset(10000, 784)
test_dataset(10000, 784)


In [16]:
num_steps = 3001
print('num steps: ' + str(num_steps))
scores = {}

for kp in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    for lr in [0.1, 0.001, 0.0001]:
        print(">> with keep prob of " + str(kp))
        print(">> with lr  " + str(lr))
        with tf.Session(graph=reluGraph) as session:
          tf.global_variables_initializer().run()
          print("Initialized - " + str(num_steps))
#           writer = tf.summary.FileWriter("/Users/santiago/code/tensorflow/projects/udacity/", session.graph)
          for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {
                tf_train_dataset : batch_data, 
                tf_train_labels : batch_labels, 
                keep_prob : kp,
                learnr : lr
            }
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
          acc = accuracy(test_prediction.eval(), test_labels)
          scores[(kp, lr)] = acc
          print("Test accuracy: %.1f%%" % acc)
print(scores)

num steps: 3001
>> with keep prob of 0.5
>> with lr  0.1
Initialized - 3001
Minibatch loss at step 0: 10200.326172
Minibatch accuracy: 7.8%
Validation accuracy: 11.2%
Minibatch loss at step 500: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 4.7%
Validation accuracy: 10.0%
Minibatch loss at step 1500: nan
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Test accuracy: 10.0%
>> with keep prob of 0.5
>> with lr  0.001
Initialized - 3001
Minibatch loss at step 0: 12552.324219
Minibatch accuracy: 8.6%
Validation accuracy: 10.6%
Minibatch loss at step 500: 1754.590576
Minibatch accuracy: 58.6%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 1334.133789
Minibatch accur

Validation accuracy: 75.4%
Minibatch loss at step 1500: 1703.451904
Minibatch accuracy: 67.2%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 1555.818115
Minibatch accuracy: 61.7%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 1456.937500
Minibatch accuracy: 68.0%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 1012.523315
Minibatch accuracy: 75.8%
Validation accuracy: 79.1%
Test accuracy: 87.0%
>> with keep prob of 0.9
>> with lr  0.1
Initialized - 3001
Minibatch loss at step 0: 4757.179199
Minibatch accuracy: 15.6%
Validation accuracy: 23.2%
Minibatch loss at step 500: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 4.7%
Validation accuracy: 10.0%
Minibatch loss at step 1500: nan
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
